# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,smithers,54.7804,-127.1743,15.18,47,75,1.54,CA,1697329724
1,1,port mathurin,-19.6833,63.4167,22.67,69,0,4.78,MU,1697329724
2,2,adamstown,-25.0660,-130.1015,21.38,78,97,2.61,PN,1697329725
3,3,qaqortoq,60.7167,-46.0333,3.70,68,18,1.32,GL,1697329427
4,4,port-aux-francais,-49.3500,70.2167,0.38,69,84,12.26,TF,1697329725


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria 
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']>=21)]
ideal_city_df = ideal_city_df.loc[(city_data_df['Max Temp']<29)]
ideal_city_df = ideal_city_df.loc[(city_data_df['Cloudiness']==0)]
ideal_city_df = ideal_city_df.loc[(city_data_df['Wind Speed']<5)]

# Drop any rows with null values
ideal_city_df.dropna()

# Reset index values
ideal_city_df = ideal_city_df.reset_index(drop=True)

# Display sample data
ideal_city_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,port mathurin,-19.6833,63.4167,22.67,69,0,4.78,MU,1697329724
1,19,hawaiian paradise park,19.5933,-154.9731,28.91,75,0,4.12,US,1697329728
2,40,mermoz boabab,14.7065,-17.4758,28.88,84,0,2.06,SN,1697329735
3,88,kerrville,30.0474,-99.1403,21.99,34,0,2.06,US,1697329686
4,114,al kharijah,25.4514,30.5464,23.08,35,0,2.97,EG,1697329749
5,118,forster,-32.1667,152.5167,22.94,52,0,2.77,AU,1697329750
6,131,formosa,-26.1775,-58.1781,22.20,72,0,3.09,AR,1697329648
7,134,tazacorte,28.6290,-17.9293,28.42,24,0,4.92,ES,1697329753
8,147,as sulayyil,20.4607,45.5779,26.14,22,0,2.01,SA,1697329453
9,200,byron bay,-28.6500,153.6167,28.73,40,0,2.27,AU,1697329763


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,69,
1,hawaiian paradise park,US,19.5933,-154.9731,75,
2,mermoz boabab,SN,14.7065,-17.4758,84,
3,kerrville,US,30.0474,-99.1403,34,
4,al kharijah,EG,25.4514,30.5464,35,
5,forster,AU,-32.1667,152.5167,52,
6,formosa,AR,-26.1775,-58.1781,72,
7,tazacorte,ES,28.6290,-17.9293,24,
8,as sulayyil,SA,20.4607,45.5779,22,
9,byron bay,AU,-28.6500,153.6167,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [53]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 10
    params = {
            "categories":categories,
            "limit":limit,
            "filter":filters,
            "bias":bias,
            "apiKey":geoapify_key    
            }
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary and convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
port mathurin - nearest hotel: Escale Vacances
hawaiian paradise park - nearest hotel: No hotel found
mermoz boabab - nearest hotel: Le Shéhérazade
kerrville - nearest hotel: Inn Of The Hills
al kharijah - nearest hotel: Kharga Hotel
forster - nearest hotel: No hotel found
formosa - nearest hotel: Hotel Hostal del Rey
tazacorte - nearest hotel: App Leyma
as sulayyil - nearest hotel: No hotel found
byron bay - nearest hotel: Lord Byron Resort
bisenda buzurg - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
rabigh - nearest hotel: S.A. TALKE Villa
hun - nearest hotel: فندق الهروج
mundra - nearest hotel: Sapphire
glen innes - nearest hotel: No hotel found
benjamin hill - nearest hotel: No hotel found
inaruwa - nearest hotel: Hotel Royal
nahiyat al fuhud - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
ghadamis - nearest hotel: فندق عين الفارس
le vauclin - nearest hotel: Hôtel Macabou
coffs harbour - nearest hotel: The 

In [55]:
# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,69,Escale Vacances
1,hawaiian paradise park,US,19.5933,-154.9731,75,No hotel found
2,mermoz boabab,SN,14.7065,-17.4758,84,Le Shéhérazade
3,kerrville,US,30.0474,-99.1403,34,Inn Of The Hills
4,al kharijah,EG,25.4514,30.5464,35,Kharga Hotel
5,forster,AU,-32.1667,152.5167,52,No hotel found
6,formosa,AR,-26.1775,-58.1781,72,Hotel Hostal del Rey
7,tazacorte,ES,28.6290,-17.9293,24,App Leyma
8,as sulayyil,SA,20.4607,45.5779,22,No hotel found
9,byron bay,AU,-28.6500,153.6167,40,Lord Byron Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
# Drop cities with no hotel found.
hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!='No hotel found')]
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,69,Escale Vacances
2,mermoz boabab,SN,14.7065,-17.4758,84,Le Shéhérazade
3,kerrville,US,30.0474,-99.1403,34,Inn Of The Hills
4,al kharijah,EG,25.4514,30.5464,35,Kharga Hotel
6,formosa,AR,-26.1775,-58.1781,72,Hotel Hostal del Rey
7,tazacorte,ES,28.6290,-17.9293,24,App Leyma
9,byron bay,AU,-28.6500,153.6167,40,Lord Byron Resort
11,alice springs,AU,-23.7000,133.8833,12,Aurora Alice Springs
12,rabigh,SA,22.7986,39.0349,71,S.A. TALKE Villa
13,hun,LY,29.1268,15.9477,36,فندق الهروج


In [87]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    hover_cols=['City', 'Hotel Name'],
    color = "Country",
    title = "Vacation Spots"
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,City,Hotel Name)